In [1]:
!pip install langchain-groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.8/332.8 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.4/127.4 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 1.6 MB/s eta 0:00:00


In [2]:
import json
import pickle
import pandas as pd
import numpy as np
from getpass import getpass
from tqdm import tqdm
from typing import List

from langchain_groq import ChatGroq
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field

In [3]:
GROQ_API_KEY = getpass("Enter your Groq API key: ")

Enter your Groq API key: ··········


In [4]:
llama3_70b_temp0 = ChatGroq(
    temperature=0,
    model_name="llama3-70b-8192",
    api_key=GROQ_API_KEY)

# Test
llama3_70b_temp0.invoke("What is the capital of France?")

AIMessage(content='The capital of France is Paris.', response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 17, 'total_tokens': 25, 'completion_time': 0.022857143, 'prompt_time': 0.005051632, 'queue_time': None, 'total_time': 0.027908775}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_87cbfbbc4d', 'finish_reason': 'stop', 'logprobs': None}, id='run-afc029cd-c449-44fd-a537-ee406874b12d-0')

In [6]:
df = pd.read_csv("/content/dataset_concurso_rag.csv")
df.columns, len(df)

(Index(['pergunta', 'resposta', 'comentarios', 'numero_na_captura', 'id',
        'area', 'ano', 'banca', 'prova', 'duvidou_do_gabarito', 'enunciado',
        'assertivas', 'alternativas', 'classificacao',
        'artigos_nos_comentarios', 'artigos_completos', 'artigo_inventado',
        'topicos_para_busca', 'artigos_candidatos', 'frase_assunto_da_questao',
        'rag_art_inventado_full', 'rag_art_inventado_full_sem_art',
        'rag_assunto_full', 'rag_topicos_full', 'rag_artigos_candidatos',
        'rag_artigos_candidatos_grupos'],
       dtype='object'),
 37)

In [7]:
df[['rag_art_inventado_full', 'rag_art_inventado_full_sem_art', 'rag_assunto_full', 'rag_topicos_full', 'rag_artigos_candidatos', 'rag_artigos_candidatos_grupos']]

,rag_art_inventado_full,rag_art_inventado_full_sem_art,rag_assunto_full,rag_topicos_full,rag_artigos_candidatos,rag_artigos_candidatos_grupos
0,"Art. 1.238. Aquele que, por quinze anos, sem i...","Art. 1.238. Aquele que, por quinze anos, sem i...","Art. 1.824. O herdeiro pode, em ação de petiçã...",Art. 1.210. O possuidor tem direito a ser mant...,Art. 1.815. A exclusão do herdeiro ou legatári...,Art. 1.814. São excluídos da sucessão os herde...
1,Art. 1.976. O testador pode nomear um ou mais ...,Art. 1.976. O testador pode nomear um ou mais ...,Art. 26. Decorrido um ano da arrecadação dos b...,"Art. 1.784. Aberta a sucessão, a herança trans...",Art. 1.862. São testamentos ordinários:I - o p...,Art. 1.862. São testamentos ordinários:\nI - o...
2,Art. 1.876. O testamento particular pode ser e...,Art. 1.876. O testamento particular pode ser e...,"Art. 1.857. Toda pessoa capaz pode dispor, por...",Art. 212. Salvo o negócio a que se impõe forma...,"Art. 1.857. Toda pessoa capaz pode dispor, por...",Art. 1.862. São testamentos ordinários:\nI - o...
3,Art. 1.976. O testador pode nomear um ou mais ...,Art. 1.976. O testador pode nomear um ou mais ...,Art. 1.976. O testador pode nomear um ou mais ...,Art. 92. Principal é o bem que existe sobre si...,"Art. 1.848. Salvo se houver justa causa, decla...","Art. 1.860. Além dos incapazes, não podem test..."
4,Art. 1.723. É reconhecida como entidade famili...,Art. 1.723. É reconhecida como entidade famili...,Art. 1.723. É reconhecida como entidade famili...,Art. 1.723. É reconhecida como entidade famili...,Art. 1.723. É reconhecida como entidade famili...,"Art. 1.784. Aberta a sucessão, a herança trans..."
5,"Art. 1.022. A sociedade adquire direitos, assu...","Art. 1.022. A sociedade adquire direitos, assu...","Art. 1.022. A sociedade adquire direitos, assu...",Art. 986. Enquanto não inscritos os atos const...,"Art. 1.793. O direito à sucessão aberta, bem c...",Art. 1.791. A herança defere-se como um todo u...
6,"Art. 1.596. Os filhos, havidos ou não da relaç...","Art. 1.596. Os filhos, havidos ou não da relaç...","Art. 1.658. No regime de comunhão parcial, com...","Art. 1.639. É lícito aos nubentes, antes de ce...",Art. 1.576. A separação judicial põe termo aos...,"Art. 1.596. Os filhos, havidos ou não da relaç..."
7,Art. 1.997. A herança responde pelo pagamento ...,Art. 1.997. A herança responde pelo pagamento ...,"Art. 789. Nos seguros de pessoas, o capital se...","Art. 818. Pelo contrato de fiança, uma pessoa ...",Art. 1.792. O herdeiro não responde por encarg...,Art. 1.791. A herança defere-se como um todo u...
8,Art. 26. Decorrido um ano da arrecadação dos b...,Art. 26. Decorrido um ano da arrecadação dos b...,"Art. 1.824. O herdeiro pode, em ação de petiçã...","Art. 1.824. O herdeiro pode, em ação de petiçã...",Art. 1.813. Quando o herdeiro prejudicar os se...,Art. 1.814. São excluídos da sucessão os herde...
9,"Art. 1.857. Toda pessoa capaz pode dispor, por...","Art. 1.857. Toda pessoa capaz pode dispor, por...",Art. 26. Decorrido um ano da arrecadação dos b...,Art. 1.798. Legitimam-se a suceder as pessoas ...,Art. 1.830. Somente é reconhecido direito suce...,Art. 1.845. São herdeiros necessários os desce...


In [8]:
def loop_fn(df, fn, kwarg=None):
    list_of_outputs = []
    for index, row in tqdm(df.iterrows(), total=len(df)):
        try:
            if kwarg:
                output = fn(row, kwarg)
            else:
                output = fn(row)
            print(output)
        except Exception as e:
            print(f"Error processing row {index}: {e}")
            output = "Error"
        list_of_outputs.append(output)
    return list_of_outputs

In [9]:
class GET_CONTEXT_JSON(BaseModel):
    analise: str = Field(description="Análise da questão, resposta e artigos de contexto.")
    parte_dos_artigos: List[str] = Field(description="Porçao ou trecho do artigo que ajuda a responder a pergunta.")
    artigos_id: List[str] = Field(description="Nome do artigos cujos trechos respondem a pergunta.")
    justificativa_de_cada_trecho: List[str] = Field(description="Razões pelas quais os trechos ajudam a responder a pergunta.")
    informacao_faltante: str = Field(description="Informação faltante que impede a resolução correta da questão, se houver.")
    resposta_obtida: str = Field(description="Resposta obtida pela análise dos trechos selecionados.")
    alternativa_resposta_obtida: str = Field(description="Alternativa da resposta obtida: A, B, C, D ou E.")
    resposta_correta: str = Field(description="Sim, se a resposta obtida for igual a resposta oficial.")


output_parser = JsonOutputParser(pydantic_object=GET_CONTEXT_JSON)

In [10]:
prompt = """
Você é professor de Direito e ávido conhecedor do Código Civil Brasileiro.
Use seus conhecimentos em leis e códigos analisar os itens da questão.

Em seguida selecione trechos dos artigos que melhor resolvem as sentenças da questão.

Irei fornecer uma lista de possíveis artigos do Código Civil e uma outra lista de artigos
citados por estudantes.

Os trechos devem ser usados para CONFIRMAR ou NEGAR as sentenças da questão.

Depois de gerar os trechos, análise se eles ajudam a chegar na resposta correta.
Caso não possibilitem chegar na mesma resposta, indique que informação está faltando.

questão: {questão}
resposta correta: {resposta}
artigos estudantes: {artigos_estudantes}
artigos possíveis: {artigos_da_busca}

Gere somente as informações dos campos do JSON, sem introduções ou formalismos.
Instruções de saída: {format_instructions}
""".strip()

prompt_template = PromptTemplate(
    input_variables=["questão", "resposta", "artigos_estudantes", "artigos_da_busca"],
    template=prompt,
    partial_variables={"format_instructions": output_parser.get_format_instructions()},
)

chain = prompt_template | llama3_70b_temp0 | output_parser

In [12]:
def call_context_generator(sample, rag_column):
    output = chain.invoke(
        {"questão": sample['pergunta'],
         "resposta": sample['resposta'],
         "artigos_estudantes": sample['artigos_completos'],
         "artigos_da_busca": sample[rag_column],
         })
    return output

# test
out = call_context_generator(df.iloc[8], rag_column='rag_topicos_full')
out

{'analise': 'A questão apresenta uma situação em que Diana, devedora do Banco XYZ, renuncia à herança deixada por sua tia Judith para evitar que o banco se aproprie da herança. O Banco XYZ pode aceitar a herança em nome de Diana, desde que obtenha autorização judicial.',
 'parte_dos_artigos': ['§ 1 o A habilitação dos credores se fará no prazo de trinta dias seguintes ao conhecimento do fato.',
  '§ 2 o Pagas as dívidas do renunciante, prevalece a renúncia quanto ao remanescente, que será devolvido aos demais herdeiros.'],
 'artigos_id': ['Art. 1.813'],
 'justificativa_de_cada_trecho': ['O artigo 1.813 estabelece que os credores podem aceitar a herança em nome do renunciante, desde que obtenham autorização judicial.',
  'O § 1 o do art. 1.813 estabelece o prazo de 30 dias para a habilitação dos credores.',
  'O § 2 o do art. 1.813 estabelece que, após o pagamento das dívidas do renunciante, a renúncia prevalece quanto ao remanescente, que será devolvido aos demais herdeiros.'],
 'infor

In [13]:
list_of_outputs = loop_fn(df, call_context_generator, kwarg='rag_topicos_full')

  3%|▎         | 1/37 [00:42<25:26, 42.39s/it]

{'analise': 'A questão apresenta um caso de herança, onde Diego, herdeiro aparente e de boa-fé, tem direito a frutos e benfeitorias. O art. 1.826 do Código Civil estabelece que o possuidor da herança está obrigado à restituição dos bens do acervo, fixando-se-lhe a responsabilidade segundo a sua posse.', 'parte_dos_artigos': ['Art. 1.826', 'Art. 1.214', 'Art. 1.219'], 'artigos_id': ['Art. 1.826', 'Art. 1.214', 'Art. 1.219'], 'justificativa_de_cada_trecho': ['O art. 1.826 estabelece a responsabilidade do possuidor da herança.', 'O art. 1.214 define os frutos que o possuidor de boa-fé tem direito.', 'O art. 1.219 estabelece o direito do possuidor de boa-fé à indenização das benfeitorias.'], 'informacao_faltante': '', 'resposta_obtida': 'C) todos os frutos percebidos até a citação, bem como o direito à indenização das benfeitorias, mas só pode exercer o direito de retenção pelo valor das benfeitorias necessárias e úteis.', 'alternativa_resposta_obtida': 'C', 'resposta_correta': 'Sim'}


  5%|▌         | 2/37 [00:47<11:51, 20.33s/it]

{'analise': 'Análise da questão: A sucessão testamentária é um componente importante do planejamento patrimonial e permite que as pessoas tenham controle sobre o destino de seus bens após a morte.', 'parte_dos_artigos': ['Art. 1.867. Ao cego só se permite o testamento público, que lhe será lido, em voz alta, duas vezes, uma pelo tabelião ou por seu substituto legal, e a outra por uma das testemunhas, designada pelo testador, fazendo-se de tudo circunstanciada menção no testamento.', 'Art. 1.868. O testamento escrito pelo testador, ou por outra pessoa, a seu rogo, e por aquele assinado, será válido se aprovado pelo tabelião ou seu substituto legal, observadas as seguintes formalidades:...', 'Art. 1.879. Em circunstâncias excepcionais declaradas na cédula, o testamento particular de próprio punho e assinado pelo testador, sem testemunhas, poderá ser confirmado, a critério do juiz.', 'Art. 1.880. O testamento particular pode ser escrito em língua estrangeira, contanto que as testemunhas a

  8%|▊         | 3/37 [01:29<17:14, 30.43s/it]

...

{'analise': 'A legítima dos herdeiros necessários não pode ser incluída no testamento.', 'parte_dos_artigos': ['§ 1 o A legítima dos herdeiros necessários não poderá ser incluída no testamento.'], 'artigos_id': ['Art. 1.857'], 'justificativa_de_cada_trecho': ['Este artigo explicitamente proíbe a inclusão da legítima dos herdeiros necessários no testamento.'], 'informacao_faltante': '', 'resposta_obtida': 'Certo', 'alternativa_resposta_obtida': 'C', 'resposta_correta': 'Sim'}


 97%|█████████▋| 36/37 [21:26<00:42, 42.42s/it]

{'analise': 'A questão refere-se à prescrição da petição de herança em relação à abertura da sucessão.', 'parte_dos_artigos': ['Art. 1.822. A declaração de vacância da herança não prejudicará os herdeiros que legalmente se habilitarem; mas, decorridos cinco anos da abertura da sucessão, os bens arrecadados passarão ao domínio do Município ou do Distrito Federal, se localizados nas respectivas circunscrições, incorporando-se ao domínio da União quando situados em território federal.'], 'artigos_id': ['Art. 1.822'], 'justificativa_de_cada_trecho': ['Este artigo estabelece que, após 5 anos da abertura da sucessão, os bens arrecadados passarão ao domínio do Município ou do Distrito Federal, o que implica que a petição de herança está prescrita.'], 'informacao_faltante': '', 'resposta_obtida': 'E) está prescrita, porque já se consumou o prazo prescricional, que deve ser contado da abertura da sucessão.', 'alternativa_resposta_obtida': 'E', 'resposta_correta': 'Sim'}


100%|██████████| 37/37 [22:09<00:00, 35.92s/it]

{'analise': 'A questão apresenta uma situação hipotética em que João, nascido em Brasília - DF, viveu toda a sua vida em Penha - SC e morreu em Florianópolis - SC, e sua esposa Ana estava grávida quando ele morreu. A pergunta é se o nascituro tem legitimidade para suceder.', 'parte_dos_artigos': ['Legitimam-se a suceder as pessoas nascidas ou já concebidas no momento da abertura da sucessão.'], 'artigos_id': ['Art. 1.798'], 'justificativa_de_cada_trecho': ['Este artigo estabelece que as pessoas nascidas ou concebidas no momento da abertura da sucessão têm legitimidade para suceder, o que inclui o nascituro.'], 'informacao_faltante': '', 'resposta_obtida': 'Sim, o nascituro tem legitimidade para suceder.', 'alternativa_resposta_obtida': 'C', 'resposta_correta': 'Sim'}


In [24]:
#df.iloc[21]['artigos_completos']
df.iloc[21]['rag_topicos_full']

'Art. 1.639. É lícito aos nubentes, antes de celebrado o casamento, estipular, quanto aos seus bens, o que lhes aprouver.\n\n§ 1 o O regime de bens entre os cônjuges começa a vigorar desde a data do casamento.\n\n§ 2 o É admissível alteração do regime de bens, mediante autorização judicial em pedido motivado de ambos os cônjuges, apurada a procedência das razões invocadas e ressalvados os direitos de terceiros.\n\n  \nArt. 1.640. Não havendo convenção, ou sendo ela nula ou ineficaz, vigorará, quanto aos bens entre os cônjuges, o regime da comunhão parcial.\n\nParágrafo único. Poderão os nubentes, no processo de habilitação, optar por qualquer dos regimes que este código regula. Quanto à forma, reduzir-se-á a termo a opção pela comunhão parcial, fazendo-se o pacto antenupcial por escritura pública, nas demais escolhas.\n\n  \nArt. 1.641. É obrigatório o regime da separação de bens no casamento:\n\nI - das pessoas que o contraírem com inobservância das causas suspensivas da celebração do

In [28]:
copy_21 = df.iloc[21].copy()
copy_21['artigos_completos'] = """'Art. 1.830. Somente é reconhecido direito sucessório ao cônjuge sobrevivente se, ao tempo da morte do outro, não estavam separados judicialmente, nem separados de fato há mais de dois anos, salvo prova, neste caso, de que essa convivência se tornara impossível sem culpa do sobrevivente.', 'Art. 25. O cônjuge do ausente, sempre que não esteja separado judicialmente, ou de fato por mais de dois anos antes da declaração da ausência, será o seu legítimo curador.§ 1º Em falta do cônjuge, a curadoria dos bens do ausente incumbe aos pais ou aos descendentes, nesta ordem, não havendo impedimento que os iniba de exercer o cargo.§ 2º Entre os descendentes, os mais próximos precedem os mais remotos.§ 3º Na falta das pessoas mencionadas, compete ao juiz a escolha do curador.'"""
copy_21['rag_topicos_full'] = df.iloc[21]['rag_topicos_full'].replace('\n\n', ' ').replace('  ', ' ').strip().split('Art. 50. ')[0].strip()
out = call_context_generator(copy_21, rag_column='rag_topicos_full')
out

{'analise': 'A questão está relacionada ao direito de sucessão e ao regime de bens entre os cônjuges.',
 'parte_dos_artigos': ['Art. 1.830.'],
 'artigos_id': ['Art. 1.830'],
 'justificativa_de_cada_trecho': ['Este artigo estabelece que o cônjuge sobrevivente tem direito sucessório se, ao tempo da morte do outro, não estavam separados judicialmente, nem separados de fato há mais de dois anos, salvo prova, neste caso, de que essa convivência se tornara impossível sem culpa do sobrevivente.'],
 'informacao_faltante': '',
 'resposta_obtida': 'A',
 'alternativa_resposta_obtida': 'A',
 'resposta_correta': 'Sim'}

In [29]:
list_of_outputs[21] = out

In [31]:
#df.iloc[33]['artigos_completos']
df.iloc[33]['rag_topicos_full']

'Art. 1.694. Podem os parentes, os cônjuges ou companheiros pedir uns aos outros os alimentos de que necessitem para viver de modo compatível com a sua condição social, inclusive para atender às necessidades de sua educação.\n\n§ 1 o Os alimentos devem ser fixados na proporção das necessidades do reclamante e dos recursos da pessoa obrigada.\n\n§ 2 o Os alimentos serão apenas os indispensáveis à subsistência, quando a situação de necessidade resultar de culpa de quem os pleiteia.\n\n  \nArt. 1.695. São devidos os alimentos quando quem os pretende não tem bens suficientes, nem pode prover, pelo seu trabalho, à própria mantença, e aquele, de quem se reclamam, pode fornecê-los, sem desfalque do necessário ao seu sustento.\n\n  \nArt. 1.696. O direito à prestação de alimentos é recíproco entre pais e filhos, e extensivo a todos os ascendentes, recaindo a obrigação nos mais próximos em grau, uns em falta de outros.\n\n  \nArt. 1.697. Na falta dos ascendentes cabe a obrigação aos descendente

In [40]:
copy_33 = df.iloc[33].copy()
copy_33['rag_topicos_full'] = df.iloc[33]['rag_topicos_full'].replace('\n\n', ' ').replace('  ', ' ').replace(' \n', ' ').strip().split('Art. 4º')[0].strip()
out = call_context_generator(copy_33, rag_column='rag_topicos_full')
out

{'analise': 'Julgada procedente a investigação de paternidade, os alimentos são devidos a partir da citação.',
 'parte_dos_artigos': ['Art. 1.696. O direito à prestação de alimentos é recíproco entre pais e filhos, e extensivo a todos os ascendentes, recaindo a obrigação nos mais próximos em grau, uns em falta de outros.',
  'Art. 1.698. Se o parente, que deve alimentos em primeiro lugar, não estiver em condições de suportar totalmente o encargo, serão chamados a concorrer os de grau imediato; sendo várias as pessoas obrigadas a prestar alimentos, todas devem concorrer na proporção dos respectivos recursos, e, intentada ação contra uma delas, poderão as demais ser chamadas a integrar a lide.'],
 'artigos_id': ['Art. 1.696', 'Art. 1.698'],
 'justificativa_de_cada_trecho': ['O art. 1.696 estabelece que o direito à prestação de alimentos é recíproco entre pais e filhos, e extensivo a todos os ascendentes, recaindo a obrigação nos mais próximos em grau, uns em falta de outros.',
  'O art. 

In [41]:
list_of_outputs[33] = out

In [42]:
rag_topicos_full = pd.DataFrame(list_of_outputs)
rag_topicos_full.to_csv("rag_topicos_full.csv", index=False)

In [43]:
rag_topicos_full

,analise,parte_dos_artigos,artigos_id,justificativa_de_cada_trecho,informacao_faltante,resposta_obtida,alternativa_resposta_obtida,resposta_correta
0,"A questão apresenta um caso de herança, onde D...","[Art. 1.826, Art. 1.214, Art. 1.219]","[Art. 1.826, Art. 1.214, Art. 1.219]",[O art. 1.826 estabelece a responsabilidade do...,,"C) todos os frutos percebidos até a citação, b...",C,Sim
1,Análise da questão: A sucessão testamentária é...,[Art. 1.867. Ao cego só se permite o testament...,"[Art. 1.867, Art. 1.868, Art. 1.879, Art. 1.880]",[O artigo 1.867 confirma que ao cego só se per...,,As afirmativas I e IV estão corretas.,B,Sim
2,A questão apresenta um caso de testamento feit...,"[Art. 1.860. Além dos incapazes, não podem tes...","[Art. 1.860, Art. 1.858, Art. 1.876, Art. 1.880]",[O art. 1.860 estabelece que os maiores de 16 ...,A idade exata do menor de 16 anos que fez o te...,"O testamento não é válido, pois o menor de 16 ...",B,Sim
3,A questão apresenta uma situação hipotética em...,"[Art. 1.860, Art. 1.799, Art. 1.900, Art. 1.80...","[Art. 1.860, Art. 1.799, Art. 1.900, Art. 1.80...",[O Art. 1.860 estabelece que a incapacidade re...,,A incapacidade relativa de Mariana no momento ...,C,Sim
4,A questão apresenta uma situação hipotética em...,[§ 1 o A união estável não se constituirá se o...,[Art. 1.723],[Este trecho do artigo 1.723 estabelece que a ...,,"Sim, é possível configurar uma união estável e...",C,Sim
5,A questão discute a responsabilidade dos herde...,[nenhum destes será obrigado a pagar senão a q...,[Art. 276],[Este trecho do art. 276 explica que os herdei...,,E,E,Sim
6,A questão apresentada refere-se ao divórcio cu...,"[Art. 1.639. É lícito aos nubentes, antes de c...","[Art. 1.639, Art. 1.642, Art. 1.647, Art. 1.64...",[O regime de bens adotado pelo casal é o da co...,Não há informação faltante.,A resposta obtida é a letra E.,E,Sim
7,A questão discute a responsabilidade solidária...,[Se um dos credores solidários falecer deixand...,[Art. 270],"[Este artigo estabelece que, em caso de faleci...",,Errado,E,Sim
8,"A questão apresenta uma situação em que Diana,...",[§ 1 o A habilitação dos credores se fará no p...,[Art. 1.813],[O artigo 1.813 estabelece que os credores pod...,,O Banco XYZ pode aceitar a herança em nome de ...,D,Sim
9,Análise da questão: A alternativa B é a corret...,[Art. 1.858. O testamento é ato personalíssimo...,[Art. 1.858],[Este artigo explicitamente afirma que o testa...,,"B) O testamento é ato personalíssimo, podendo ...",B,Sim
